# Nichtlineare Optimierung für reelle Funktionen

__Manfred Brill, Hochschule Kaiserslautern__

## Aufgabenstellung
Für eine Funktion f einer reellen Veränderlichen suchen wir ein lokales Minimum. Solche Fragestellungen habenw wir bereits bearbeitet und bei Abstiegsverfahren für die Berechnung eines lokalen Minimums eines skalaren Felds müssen wir diese Aufgabe ebenfalls lösen.

Man könnte auf die Idee kommen die Funktion f abzuleiten und eine Nullstelle dieser ersten Ableitung zu bestimmen. Verfahren für die Näherung von Nullstellen finden wir in SciPy. Aber dann müssen wir auch die zweite Ableitung aufstellen und die hinvereichende Bedingung überprüfen. Erfolgsversprechender ist es Verfahren einzusetzen, die ein Minimum annähern. Dabei gibt es Verfahren, die Informationen über die Ableitung verwenden. Aber es gibt auch sogenannte *ableitungsfreie* Verfahren, die außer der Möglichkeit, die Funktion auswerten zu können Informationen über geeignete Startwerde benötigen.


## Verfahren
Alle Verfahnren die wir betrachten gehen von einer unimodalen Funktion aus. Damit ist gemeint, dass wir ein Intervall kennen von dem wir wissen, dass in diesem Intervall exakt ein lokales Minimum vorliegt. Ähnlich wie die Bisektion für die Näherung einer Nullstelle für eine stetige Funktion unterteilen wir dieses Intervall [a, b] und berechnen zwei x-Werte im Innern von  [a, b]. Dann untersuchen wir die Funktionswerte an diesen neuen Stellen und vergleichen sie mit den Funktionswerten von a und b. Wir wählen ein neues Intervall aus, das kleiner ist als vorher und iterieren so lange, bis wir ein Abbruchkriterium erfüllen.

Als optimal haben sich hier der goldene Schnitt für die Unterteilung des Intervalls herausgestellt. Das Verfahren, das diesen Ansatz verwendet nennen wir *golden section search*. Der goldene Schnitt ist eine irratinoale Zahl, so dass man häufig ausnutzt, dass wir diese Zahl als Quotient von Fibonacci-Zahlen annäherung können. Daraus entsteht daas Verfahren *Fibonacci Search*.

Ein weitere Ansatz, der in der Praxis auch häufig zu guten Ergebnissen führt ist die Idee, die Funktion f die wir minimieren möchten mit Hilfe eines quadratischen Polynoms anzunähern. Ist f im Intervall [a, b] unumodal, dann können wir das lokale Minimum des quadratrischen Polynoms bestimmtn und anschließend wieder das Intervall, von dem wir wissen, dass das lokale Minimum von f darin liegt, zu verkleinern. Vorteil dieses Verfahrens ist, dass wir drei Funktionsauswerten statt vier bei Golden Section Search benötigen.

Der Brent-Algorithmus implementiert eine *safe guarded quadratic interpolation* und kombiniert beide Ansätze. Damit entsteht ein sehr stabiler Algorithmus, der als *state-of-the-art* angesehen werden kann.

## Realisierung in Scipy
In SciPy finden wir die Funktion *optimize.minimize_scalar*. Mit Hilfe des Parameters *method* können wir wählen, ob wir Golden Section Search oder Brent einsetzen möchten. Vorausgesetzt wird, dass wir eine Python-Funktion implementiert haben die Funktionswerte für die Zielfunktion f berechnet. Die Funktion bietet die Möglichkeit ein *bracket* zu übergeben. Damit sind drei x-Werte a < c < b gemeint von denen wir wissen, dass

f(a) > f(c), f(b) > f(c)

erfüllt ist. Im Intervall [a, b] ist f also unimodal.

In [1]:
import numpy as np
from scipy import optimize

def f(x):
    """
    Funktion, die minimiert werden soll
    """
    return (np.exp(x) - 2.5*x*x + x - 1.0)

Wir können f grafisch ausgeben, damit finden wir schnell ein Bracket mit a=0, c=2 und b=3.5. Klar ist - die Lösung die SciPy berechnet wird im Intervall [a, b] liegen. Wir überprüfen ob wir ein Bracket gefunden haben, um zu vermweiden, dass SciPy eine Fehlermeldung ausgibt.

In [2]:
print('f(0)=', f(0.0))
print('f(2)=', f(2.0))
print('f(3.5)=', f(3.5))

f(0)= 0.0
f(2)= -1.6109439010693496
f(3.5)= 4.9904519586923115


Jetzt können wir das Verfahren aufrufen. Dabei verwenden den Brent-Algorithmus.

In [5]:
result = optimize.minimize_scalar(f,
                                  method='brent',
                                  bracket=(0.0, 2.0, 3.5),
                                  tol=np.finfo(1.).eps,
                                  options={'maxiter': 50})

if result.success:
    print('Ergebnisse des Brent-Algorithmus')
    print('Der berechnete x-Wert des lokalen Minimums:', result.x)
    print('Der Funktionswert an diesem x-Wert:', result.fun)
    print(result.nfev, 'Funktionsauswertungen')
    print(result.nit, 'Iterationen')

Ergebnisse des Brent-Algorithmus
Der berechnete x-Wert des lokalen Minimums: 2.396138520421771
Der Funktionswert an diesem x-Wert: -1.9768683142947223
31 Funktionsauswertungen
27 Iterationen


Jetzt können wir das Verfahren aufrufen. Wir verwenden jetzt Golden Section Search. Dabei müssen wir die maximale Anzahl der Iterationen erhöhen. Man sieht an Hand dieses Beispiels gut, dass die Kombination zwischen quuadratischer Interpolation und Intervallteilung Vorteile hat.

In [10]:
result = optimize.minimize_scalar(f,
                                  method='golden',
                                  bracket=(0.0, 2.0, 3.5),
                                  tol=np.finfo(1.).eps,
                                  options={'maxiter': 100})

if result.success:
    print('Ergebnisse der Golden Section Search')
    print('Der berechnete x-Wert des lokalen Minimums:', result.x)
    print('Der Funktionswert an diesem x-Wert:', result.fun)
    print(result.nfev, 'Funktionsauswertungen')
    print(result.nit, 'Iterationen')

Ergebnisse der Golden Section Search
Der berechnete x-Wert des lokalen Minimums: 2.3961385102724924
Der Funktionswert an diesem x-Wert: -1.9768683142947223
80 Funktionsauswertungen
75 Iterationen


## Anwendung auf die leminskaite von Gerono
Suchen wir minimale oder maximale x- und y-Werte der Spur einer ebenen Parameterkurve wie der Leminskate von Gerono können wir die beschriebenen
Verfahren gut einsetzen. Wir suchen im Beispiele nach einem lokalen Minimum der y-Koordinaten der Spur. 
Dann verwenden wir die Funktion in der Definition
der Lemmiskate, die für die y-Koordinaten verantwortlich ist. Wir verwenden hier kein Bracket sondern beschränken uns darauf ein Intervall anzugeben in dem wir das Minimum vermuten.

In [18]:
def g(t):
    """
    Funktion für die y-Koordinaten der lemniskate von Gerono
    """
    return np.sin(t)*np.cos(t)

a = 1.25*np.pi
b = 2.0*np.pi

result = optimize.minimize_scalar(g,
                                  method='brent',
                                  bracket=(a, b),
                                  tol=np.finfo(1.).eps,
                                  options={'maxiter': 50})

if result.success:
    print('\nLokales Minimum für die y-Koordinaten der Lemniskate von Gerono')
    print('Der berechnete t-Wert des lokalen Minimums:', result.x)
    print('Der Funktionswert an diesem t-Wert:', result.fun)
    print('Die exakte Lösung lautet t =', 1.75*np.pi)
    print('Relativer Fehler:',
          np.abs((result.x-1.75*np.pi)/1.75*np.pi))


Lokales Minimum für die y-Koordinaten der Lemniskate von Gerono
Der berechnete t-Wert des lokalen Minimums: 5.497787144430887
Der Funktionswert an diesem t-Wert: -0.5000000000000001
Die exakte Lösung lautet t = 5.497787143782138
Relativer Fehler: 1.1646307819549499e-09


Wir suchen vergeblich nach Funktionen in SciPy mit denen wir ein lokales Maximum berechnen können. Das führen wir
mit Hilfe von -g durch, also der mit -1 multiplizierten Zielfunktion. Damit suchen wir nach lokalen Maxima für die y-Koordinaten
der Lemniskate von Gerono.

In [19]:
def gminus(t):
    """
    Funktion für die y-Koordinaten der lemniskate von Gerono,
    mit -1 multipliziert für lokale Maxima
    """
    return -np.sin(t)*np.cos(t)

a = 0.0
b = 1.2

print('\nLokales Maximum für die y-Koordinaten der Lemniskate von Gerono')
result = optimize.minimize_scalar(gminus,
                                  method='brent',
                                  bracket=(a, b),
                                  tol=np.finfo(1.).eps,
                                  options={'maxiter': 50})

if result.success:
    print('Der berechnete t-Wert des lokalen Maximums:', result.x)
    print('Der Funktionswert an diesem t-Wert:', result.fun)
    print('Die exakte Lösung lautet t =', 0.25*np.pi)
    print('Relativer Fehler:',
          np.abs((result.x-0.25*np.pi)/0.25*np.pi))


Lokales Maximum für die y-Koordinaten der Lemniskate von Gerono
Der berechnete t-Wert des lokalen Maximums: 0.7853981628127534
Der Funktionswert an diesem t-Wert: -0.5000000000000001
Die exakte Lösung lautet t = 0.7853981633974483
Relativer Fehler: 7.347492157451213e-09
